## Машинное обучение в бизнесе. Внедрение модели

Создание веб-сервиса на Flask, синхронизация с моделью (разработанной в W7), формирование предсказания через post-запросы к сервису

Сервис принимает новую запись и предсказывает результат прогнозирования наступления страхового случая

In [134]:
import pandas as pd
import h2o
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import json
import random
import warnings
import requests
warnings.filterwarnings('ignore')
random.seed(12345)

In [135]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 48 mins
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,12 days
H2O cluster name:,H2O_from_python_Nickel_q0jnch
H2O cluster total nodes:,1
H2O cluster free memory:,3.339 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


## Загрузка данных

In [136]:
df = pd.read_csv('ensurance_data.csv', sep=',')

# Настроечный блок

TARGET = 'LicAge'
X = df.drop(TARGET, axis=1)
DATA_COLUMNS = X.columns # все столбцы
# выборочные столбцы для тестирования сервиса
COLS = ['Exposure', 'DrivAge', 'HasKmLimit', 'BonusMalus', 'ClaimInd', 'OutUseNb', 'RiskArea']
MODEL_PATH = 'GLM_model_python_1581933695560_1' # Путь к сохраненной модели
MAIN_LINK = 'http://7be957b9.ngrok.io/predict' # Ссылка динамически меняется

In [137]:
"""Метод получения пустого H2O датафрейма под предсказание"""

def return_new_H2O_DataFrame(cols):
    l_shape = list('0' * len(cols))
    return h2o.H2OFrame([l_shape], column_names = list(cols))


"""Тестовый метод получения случайной записи"""

def get_random_one_data_from_df(df):
    # Случайный семпл в качестве входных данных для веб-сервиса
    input_seed = random.randint(0, df.shape[0])
 
    a = X.iloc[input_seed]
    b = a.values.reshape(1,-1)
    data = pd.DataFrame(b, columns = DATA_COLUMNS)
    # h2o_data = h2o.H2OFrame(data)
    
    return data


"""Метод получения предсказания"""

def get_predict(data, model_path):
    
    # Получение предсказания из случайной записи тестового датасета
    # data = get_h2o_data_from_df(data)
    
    # Загрузка модели, созданной в W7_data_preprocessing
    model_glm = h2o.load_model(model_path)
    
    # Результат работы модели
    prediction = model_glm.predict(data).as_data_frame()
    pred_value = prediction['predict'][0]
    
    return pred_value


"""Конвертация данных в json"""

def get_json_from_random_one(df):
    
    df_ = get_random_one_data_from_df(df)  
    
    # df_ = df.as_data_frame() # для приема h2o фрейма
    d = df_.to_dict(orient='records')
    j = json.dumps(d)
    res = json.loads(j)
    
    return res[0]

def map_pred_result(pred):
    pred = str(pred)
    dict_pred = {'0':'Accept', '1':'Reject'}
    res = dict_pred.get(pred)
    
    return res

In [138]:
"""Запуск веб-сервиса на Flask"""

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/predict', methods=['GET', 'POST'])
def predict_():
    try:
               
        json_input = request.json
        
        # Формирование данных для запроса на сервис
        Exposure = json_input['Exposure']
        DrivAge = json_input['DrivAge']
        HasKmLimit = json_input['HasKmLimit']
        BonusMalus = json_input['BonusMalus']        
        ClaimInd = json_input['ClaimInd']
        OutUseNb = json_input['OutUseNb']
        RiskArea = json_input['RiskArea']
        
        # h2o frame под предсказание
        hf = return_new_H2O_DataFrame(COLS)
        
        hf[0, 'Exposure'] = Exposure
        hf[0, 'DrivAge'] = DrivAge
        hf[0, 'HasKmLimit'] = HasKmLimit
        hf[0, 'BonusMalus'] = BonusMalus
        hf[0, 'ClaimInd'] = ClaimInd
        hf[0, 'OutUseNb'] = OutUseNb
        hf[0, 'RiskArea'] = RiskArea
        
        # Получение ответа от модели
        pred = int(get_predict(hf, model_path))
        
        # перевод числового предсказания в текстовый
        pred = map_pred_result(pred)
        
        # Формат ответа
        output = {
            'Exposure':Exposure,
            'DrivAge':DrivAge,
            'HasKmLimit':HasKmLimit,
            'BonusMalus':BonusMalus,
            'ClaimInd':ClaimInd,
            'OutUseNb':OutUseNb,
            'RiskArea':RiskArea,
            'Predict':pred
        }
        
        return jsonify(output)
    
    except:
      
      return "Something Goes Wrong"    


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7be957b9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


INFO:werkzeug:127.0.0.1 - - [18/Feb/2020 03:33:45] "POST /predict HTTP/1.1" 200 -


##### Тестовый запрос

In [148]:
post_data = get_json_from_random_one(df)

post_data

{'Exposure': 0.307,
 'DrivAge': 54.0,
 'HasKmLimit': 0.0,
 'BonusMalus': 50.0,
 'ClaimAmount': 0.0,
 'ClaimInd': 0.0,
 'OutUseNb': 0.0,
 'RiskArea': 6.0,
 'VehUsage_Private': 0.0,
 'VehUsage_Private+trip to office': 0.0,
 'VehUsage_Professional': 1.0,
 'VehUsage_Professional run': 0.0,
 'SocioCateg_CSP1': 0.0,
 'SocioCateg_CSP2': 0.0,
 'SocioCateg_CSP3': 0.0,
 'SocioCateg_CSP4': 1.0,
 'SocioCateg_CSP5': 0.0,
 'SocioCateg_CSP6': 0.0,
 'SocioCateg_CSP7': 0.0,
 'Exposure (^2)': 0.094249,
 'Exposure (^3)': 0.028934443,
 'Exposure (sqrt)': 0.5540758070878027,
 'Exposure (log)': -1.18090753139494,
 'Exposure (/ 2)': 0.1535,
 'Exposure (/ 4)': 0.07675,
 'DrivAge_diff1': 32.0,
 'DrivAgeSq': 2916.0,
 'ClaimsCount': 0.0}

In [149]:
"""Формирование тестового запроса в python"""

# req = requests.post(MAIN_LINK, json=post_data)
# json_response = req.json()
# print(json_response)

'Формирование тестового запроса в python'

Итоги:

- Сервису можно скармливать любые json данные - главное, чтобы поля в input заполнялись
- H2O модель корректно обрабатывает пропущенные поля, но относительно медленно работает (25 сек до ответа на запрос)